## Read CSV

In [ ]:
import pandas as pd
plans = pd.read_csv('plans.csv')
slc = pd.read_csv('slcsp.csv')
zips = pd.read_csv('zips.csv')
zips.head(10)

Remove duplicate records if any

In [ ]:
zips = zips[['zipcode', 'state', 'rate_area']].sort_values(['zipcode', 'state', 'rate_area']).drop_duplicates()

Identify duplicate zips, eg: same zipcode in multiple states / rate areas.
drop these duplicate zips so the result in the final answer is N/A

In [ ]:
dup_zips = zips[zips.duplicated(['zipcode'], keep=False)].sort_values(['zipcode', 'state', 'rate_area'])
uniq_zips = zips.drop(dup_zips.index.values)
uniq_zips.head(10)

Filter on Silver plans

In [ ]:
grp = plans[plans['metal_level'] == 'Silver'][['state', 'rate_area', 'rate']].sort_values(['state',  'rate_area', 'rate']).drop_duplicates()
grp.head(10)

Check if there are two rates for the same rate area in a state

In [ ]:
grp.groupby(['state', 'rate_area']).size().reset_index(name='counts').sort_values(['counts']).head(10)

find the index of the minimum rate plan in rate_area 
drop the lowest plans from the list

In [ ]:
lowestPlanIdx = grp.groupby(['state', 'rate_area'])['rate'].idxmin().values
grp3 = grp.drop(lowestPlanIdx)

get the second lowest rateplan

In [ ]:
grp4 = grp3.groupby(['state', 'rate_area'], as_index=False).min()

Join unique zipcode with the second lowest rate to get the final answer

In [ ]:
res1 = pd.merge(uniq_zips, grp4, on=['state', 'rate_area'], how="left")
res2 = pd.merge(slc, res1, on=['zipcode'], how="left")
ans = res2[['zipcode', 'rate_y']]
ans.head(10)